Kaggle Challenge - Fraud Detection - BGU 2019

(1) Load Data

In [1]:
from utils import *
dataset = pd.read_csv('../Detecting-Cyber-Attacks-Kaggle-Callenge/data/partial_labels.csv')
x,segments = dataset.axes
array = dataset.values
results_df = pd.DataFrame(columns=["id","label"])

(2) Pre-Processing

In [2]:
testUser=10
z = np.reshape(array[0,1:],-1)
Y_train = z[0:50]
Y_train = Y_train.astype(int)
path_read = "../Detecting-Cyber-Attacks-Kaggle-Callenge/data/FraudedRawData/User0"
all_instructions = pd.read_csv(path_read, sep=" ", header=None)
all_instructions = np.reshape(all_instructions.values,(150,100))

    #all_instructions = np.concatenate((all_instructions, arr),axis=0)

(3) Feature Engineering

In [3]:
x_train = count_vec_usr = count_vec_all = []

number_of_users = 40
number_of_segments = 150
number_of_users_with_full_information = 10

for i in range( 150 ):
    #If we are still in the first 10 users information
    if(i < 50):
        count_vec_usr.append(' '.join(all_instructions[i,:]))
        
    count_vec_all.append(' '.join(all_instructions[i,:]))

vectorizer = TfidfVectorizer(min_df=0.4,ngram_range=(1,2))
vectorizer.fit_transform(count_vec_all)

X_all = vectorizer.transform(count_vec_all)
X_all = X_all.toarray()
X_usr = vectorizer.transform(count_vec_usr)
X_usr = X_usr.toarray()

#should add also for first 10 users - the rest of the segments
unique = set()
for i in range(50):
    for j in range(100):
        unique.add(all_instructions[i][j])
    x_train.append([random.randint(0, 2)])
    
count = 0
strange = []
unique_check = set()

for i in range(50,150):
    for j in range(100):
        unique_check.add(all_instructions[i][j])
        if (all_instructions[i][j] not in unique):
            count = count + 1
    strange.append([count])
    unique_check.clear()
    count = 0

(4) Model Fit & Predict

In [4]:
x_test=np.concatenate((x_train,strange))
x_test=np.c_[X_all,x_test]
x_train=np.c_[X_usr,x_train]
clf= LocalOutlierFactor(n_neighbors=25,contamination=0.1, novelty=True)
clf.fit(x_train)
t=clf.predict(x_test)
t[t == 1] = 0
t[t == -1] = 1


ValueError: all the input arrays must have same number of dimensions

(5) Push Results to Kaggle Challenge

In [ ]:
curr_user_results = t[50:150]
curr_user_results_df = pd.DataFrame(columns=["id","label"])

start_rows = 5000
till_rows = 5100

for i in range(curr_user_results.size):
    curr_user_results_df.loc[i] = ["User{}_{}-{}".format(testUser,start_rows,till_rows),curr_user_results[i]]
    start_rows = start_rows + 100
    till_rows = till_rows + 100
results_df = results_df.append(curr_user_results_df)



In [ ]:
fname = "out/submission_results_{}.csv".format(date.today())
results_df.to_csv(fname, index=False)
os.system('kaggle competitions submit -c detectingcyberattackscallenge -f "{0}" -m "{1}"'.format(fname, "Submitting results to kaggle"))